In [42]:
# importing python libraries
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import sys
print(sys.executable)

c:\Users\shree\Downloads\Data Analyst\Python\venv\python.exe


In [2]:
#Load the Dataset
df = pd.read_csv("ecommerce_returns_synthetic_data.csv")


In [29]:
df.columns
df.describe()
df.isnull().sum()
df.count()
df.dtypes
df.drop_duplicates(inplace = True)





In [32]:
# Numerical columns 
df.fillna(df.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)

# Categorical 
df.fillna(df.select_dtypes(include='object').mode().iloc[0], inplace=True)

In [ ]:
print(df.groupby('Product_Category')['Is_Returned'].mean())
print(df.groupby('User_Gender')['Is_Returned'].mean())
print(df.groupby('User_Location')['Is_Returned'].mean())
print(df.groupby('Payment_Method')['Is_Returned'].mean())

Product_Category
Books          0.506614
Clothing       0.524500
Electronics    0.509320
Home           0.490148
Toys           0.495370
Name: Is_Returned, dtype: float64


In [45]:
df.groupby('Product_Category')['Is_Returned'].mean().to_csv('category_return_rates.csv')
features = df[['Product_Price', 'Product_Category', 'User_Location', 'Payment_Method', 'User_Gender']]
target = df['Is_Returned']

In [18]:
# Convert Date Columns
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors = 'coerce')
df['Return_Date'] = pd.to_datetime(df['Return_Date'], errors = 'coerce')
# Validate Numeric Columns
# Remove the rows with Negative or zero values in important columns
df = df[(df['Product_Price']>= 0) & 
        (df['Order_Quantity']>0)&
        (df['Discount_Applied']>=0)]
# Standardize Categrical Fields
df['Return_Status'] = df['Return_Status'].str.strip().str.title()
df['User_Gender'] = df['User_Gender'].str.strip().str.title()

# Add Return Flag
df['Is_Returned'] = df['Return_Status'] == 'Returned'

df.to_csv("Cleaned_orders_dataset.csv",index = False)
print("Dateset Cleaned and saved")

Dateset Cleaned and saved


In [20]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect("ecommerce_orders.db")

# Write the cleaned order dataset into a SQL Table
df.to_sql("orders",conn, if_exists = "replace", index = False)
print("Cleaned Data saved to SQLite database 'orders' table.")

Cleaned Data saved to SQLite database 'orders' table.


In [21]:
# Fetch top 5 most expensive orders
query = """
SELECT Order_ID, Product_Price, Order_Quantity, Product_Category
FROM Orders
ORDER BY Product_Price DESC
LIMIT 5
"""
expensive_orders = pd.read_sql_query(query,conn)
expensive_orders

,Order_ID,Product_Price,Order_Quantity,Product_Category
0,ORD00008111,499.89,1,Toys
1,ORD00009021,499.75,4,Clothing
2,ORD00002886,499.71,2,Electronics
3,ORD00009965,499.69,4,Toys
4,ORD00007098,499.68,2,Books
